<a href="https://colab.research.google.com/github/shreyanshp/colab_notebooks/blob/master/Keras_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'food_dataset/'

Mounted at /content/gdrive


In [4]:
TRAIN_DIR = base_dir
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

Found 6 images belonging to 3 classes.


In [5]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["pizza", "burger", "taco"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
NUM_EPOCHS = 1
BATCH_SIZE = 8
num_train_images = 10000

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath= root_dir + "colab_models/" +"ResNet50" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = []

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)

!rm -rf ./mymobilenet* ./model.h5
finetune_model.save('./model.h5')
    

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1250/1250 [==============================] - 451s 361ms/step - loss: 0.0259 - acc: 0.9915


KeyError: ignored

In [7]:
!pip install tensorflowjs

    100% |████████████████████████████████| 81kB 5.8MB/s 
    100% |████████████████████████████████| 96.1MB 342kB/s 
    100% |████████████████████████████████| 13.9MB 2.2MB/s 
    100% |████████████████████████████████| 358kB 17.0MB/s 
    100% |████████████████████████████████| 61kB 23.8MB/s 
    100% |████████████████████████████████| 3.0MB 9.2MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: tensorflow-hub 0.4.0
    Uninstalling tensorflow-hub-0.4.0:
      Successfully uninstalled tensorflow-hub-0.4.0


In [2]:
from google.colab import files
!tensorflowjs_converter --input_format=keras ./model.h5 ./mymobilenet
!zip -r ./mymobilenet.zip ./mymobilenet 
files.download( "./mymobilenet.zip" )

Traceback (most recent call last):
  File "/usr/local/bin/tensorflowjs_converter", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/converter.py", line 343, in main
    split_weights_by_layer=FLAGS.split_weights_by_layer)
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/converter.py", line 89, in dispatch_keras_h5_to_tensorflowjs_conversion
    model_json, groups, output_dir, quantization_dtype)
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py", line 259, in write_artifacts
    quantization_dtype=quantization_dtype)
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/write_weights.py", line 119, in write_weights
    group_bytes, total_bytes, _ = _stack_group_bytes(group)
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/write_weights.py", line 200, in _stack_group_bytes
    group_bytes_writer.write(data_bytes)
KeyboardInterrupt


KeyboardInterrupt: ignored